In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

data= fetch_openml('mnist_784', version=1)#Get data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])
def numberOfLoops(num):
    if num in ["0","6","9"]:
        return 1
    elif num == "8":
        return 2
    return 0
dfData["numLoops"] = dfData["target"].apply(numberOfLoops)
dfData["hasLoop"] = dfData["numLoops"].apply(lambda x: True if x>0 else False)

In [17]:
img_pipeline = Pipeline([("mm_scaler", MinMaxScaler())])
y = dfData[["target","numLoops","hasLoop"]]
dfData = dfData.drop(["target","numLoops","hasLoop"],axis=1)
X = dfData.copy()
X_transf = img_pipeline.fit_transform(X)

In [18]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in stratSplit.split(X_transf, y):
    X_train = X_transf[train_index]
    X_test = X_transf[test_index]
    
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]

In [4]:
log_reg = LogisticRegression(C=1e5, max_iter=100)
log_reg.fit(X_train, y_train.values)
log_reg.predict(X_test)[0]

ValueError: bad input shape (56000, 3)

In [8]:
from sklearn.multioutput import MultiOutputClassifier
y_train["target"] = y_train["target"].apply(lambda x: int(x))
y_train["hasLoop"] = y_train["hasLoop"].apply(lambda x: int(x))

# y_test["target"] = y_test["target"].apply(lambda x: int(x))
log_reg = LogisticRegression(C=1e5)
log_reg_multi_label = MultiOutputClassifier(log_reg)
log_reg_multi_label.fit(X_train, y_train.values)
log_reg_multi_label.predict(X_test)[0]

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: T

array([0, 1, 1])

In [24]:
from sklearn.base import clone

class customMultilabel():
    def __init__(self, clf):
        self.clf = clf
        self.clfs = []
        
    def fit(self, X, y):
        if len(y.shape) == 2:
            self.numOutputs = y.shape[-1]
        elif len(y.shape) == 1:
            self.numOutputs = 1
        else:
            print("Unexpected target shape")
            raise(RuntimeError)
            
        for i in range(self.numOutputs):
            self.clfs.append(clone(self.clf))
            self.clfs[i].fit(X, y.iloc[:,i])
            
    def predict(self, X):
        output = None
        for i in range(self.numOutputs):
            r = self.clfs[i].predict(X)
            if output is None:
                output = r.copy()
            else:
                output = np.c_[output, r]
            
        return output

In [25]:
cml_log_reg = customMultilabel(LogisticRegression(C=1e5))
cml_log_reg.fit(X_train, y_train)
output = cml_log_reg.predict(X_test)

/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/maxschallwig/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/

In [14]:
output[:5]

array([['0', 1, True],
       ['0', 1, True],
       ['4', 0, False],
       ['6', 1, True],
       ['1', 0, False]], dtype=object)

In [15]:
y_test[:5]

,target,numLoops,hasLoop
29205,0,1,True
5801,0,1,True
18245,4,0,False
41828,6,1,True
24289,1,0,False
